# Documentation

## Objective(s)
1. Ingest event log file data file into a consistent tabular format without error
2. Extract key features/attributes embedded in free text message log
3. Batch process the data to consolidate log files for easy of management
4. Due to the different table structure for EventList and AlarmList data (AlarmList has redundant fields removed), the preprocessing script cannot be used interchangeably when appending data to the SQL Server Table.    


# Initialisation

### Adjust Display Theme of Jupyter Notebook
Optional Step

Key Hotkeys:
1. Run cell: ctrl + enter
2. Delete Cell: DD
3. Undo Delete Cell: Z
4. Cut Cell: X
5. Copy Cell: C
6. Insert Cell Above: A
7. Insert Cell Below: B
8. Convert Cell to Code: Y
9. Convert Cell to Markdown: M

In [1]:
# install jupyterthemes
#!pip install jupyterthemes

# upgrade to latest version
#!pip install --upgrade jupyterthemes

In [2]:
#import jupyterthemes

In [3]:
# Adjust to Dark Theme
#jt -t chesterish

## Load Libraries

In [4]:
#pip install --user --upgrade pandas

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install schedule

In [6]:
# Import libraries
#import modin.pandas as pd
import pandas as pd
import numpy as np
import os
import pathlib
import datetime as dt
import time
import os # for manipulating file directories
import pyodbc # for sql operations
import sqlalchemy # for sql operations with pandas
import urllib # for defining sql connection parameters
import shutil # for transferring files between folders
#import re

## Set Options

In [7]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

In [8]:
# Choose whether to initialise db from scratch or not
initDB = True

In [9]:
# Trigger SQL Function Test
testMode = True
# Use this to reset table data especially when running tests
resetTableData = True

## Set Up Core Directories

In [10]:
# Check current directory location
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [11]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [12]:
# Create Directory for Output Files Generated
if not os.path.exists('dataCleaned'):
    os.makedirs('dataCleaned')
    
# Inspect files in directory
fileList = os.listdir()
fileList

['associationRules',
 'dataCleaned',
 'desktop.ini',
 'dummyLog',
 'HISevent_data',
 'Original Sample from 27 Oct 2020 (simplified).7z',
 'Repair Logs',
 'Sample from 27 Oct 2020 (OG).zip',
 'sample_data_allServer _(Jan2021).7z',
 'sample_data_allServer_raw&processed__(Jan2021).7z',
 'sample_data_HISevent.7z',
 'sample_data_test',
 'taggedOutput',
 'testLog',
 'testOutput']

In [13]:
# Location of Alarm and Normal Event Files
alarmLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\AlarmLogs'
eventLoc = '\\sample_data_cms\\batch 001 - 20201230 to 20210201\\EventLogs'
testLocAL = '\\sample_data_test\\batch 002\\AlarmLogs'
testLocAE = '\\sample_data_test\\batch 002\\EventLogs'

# Set variables
# Alarm Events: 'AL'  or 'TestAL' 
# All Events: 'AE' or 'TestAE' 

prefix = 'TestAL' 
batch = 'B001-'
serverEnv = 'CMS-'

## Access Files to be Processed

In [14]:
# Define root file directory folder where the files are being stored
if prefix == "AL":
    os.chdir(cwd + alarmLoc)
    #os.chdir(cwd + eventLoc)
elif prefix == "AE":
    os.chdir(cwd + eventLoc)
elif prefix == "TestAL":
    os.chdir(cwd + testLocAL)
else:
    os.chdir(cwd + testLocAE)
    
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\AlarmLogs'

In [15]:
# Get the list of all files in directory tree at given path
fileList = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    fileList += [os.path.join(dirpath, file) for file in filenames] # use this if you want to append full URL
    #fileList += filenames
    
# Inspect data
len(fileList)

762

In [16]:
testFile = fileList[0]
testFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\AlarmLogs\\AlarmListTest_1638320410_254559'

# Process Log Files

In [17]:
# Core Function for Cleaning (dependent on above functions)
# Excludes file saving
def cleaningScript_vector(fileInput, prefixInput = "TestAL", sqlServerExport = False):
    
    ##################################################################
    # Import Dependant Libraries
    ##################################################################
    
    import sys, os
    if 'pd' not in sys.modules:
        import pandas as pd
    if 'np' not in sys.modules:
        import numpy as np
    if 'dt' not in sys.modules:
        import datetime as dt
    #if 'os' not in sys.modules:
    #    import os as os
    
    ##################################################################
    # Initialise Dependant Function 1
    ##################################################################
    
    # Define function to remove free text message to facillitate formating later
    def replaceTextBetween1_series(originalTextSeries, delimeterA='Message:', delimterB='Theme:', replacementText=''):
        leadingText = originalTextSeries.str.split(delimeterA).str.get(0)
        trailingText = originalTextSeries.str.split(delimterB).str.get(1)

        return leadingText + replacementText + delimterB + trailingText
    
    
    ##################################################################
    # Initialise Dependant Function 2
    ##################################################################  
    
    def parseMessageField(originalTextSeries, delimeterA='Message:', delimterB='Theme:'):
        # Update v2.0
        # Remove leading delimiter (Delimiter A) from target string
        clip1 = originalTextSeries.str.split(delimeterA).str.get(1)
        # Remove lagging delimiter (Delimiter B) from target string
        clip2 = clip1.str.split(delimterB).str.get(0).str.strip()
        # Clean and split target string into constituent components
        clip3 = clip2.str.replace(";", "").str.split("$")
        
        
        # Delete redundant fields
        del clip1, clip2
        
        # Extract ASSET_ID_RAW from 1st Component of Clip 3
        ASSET_ID_RAW = clip3.str.get(0).str.strip()
        
        # Extract ASSET_DESCRIPTION from 2nd Component of Clip 3, a may be further delimited by a colon sign
        try:
            ASSET_DESCRIPTION = clip3.str.get(1).str.split(": ").str.get(0).str.strip()
        except: 
            ASSET_DESCRIPTION = None
        
         # Extract EVENT_DESCRIPTION from 2nd Component of Clip 3, a may be further delimited by a colon sign
        try:    
            EVENT_DESCRIPTION = clip3.str.get(1).str.split(": ").str.get(1).str.strip()
        except:
            EVENT_DESCRIPTION = None
        
         # Extract EVENT_STATUS from 3rd Component of Clip 3
        try:
            EVENT_STATUS = clip3.str.get(2).str.strip()
        except:
            EVENT_STATUS = None
        
        # Extract OPERATOR_INITIALS from 4th Component of Clip 4
        try:
            OPERATOR_INITIALS = clip3.str.get(3).str.strip()
        except:
            OPERATOR_INITIALS = None

        return ASSET_ID_RAW, ASSET_DESCRIPTION, EVENT_DESCRIPTION, EVENT_STATUS, OPERATOR_INITIALS
    
    ##################################################################
    # Initialise Dependant Function 3
    ##################################################################
    
    def parseMetaData(f_meta1, f_meta2):

        # Pre-process f_meta1
        f_meta1 = f_meta1.split('\n')[0].split('Begin Notification: ')[1].split(' ')

        # Pre-process f_meta2
        f_meta2 = f_meta2.split('End Notification: ')[1]

        # Get DATETIME_SEND
        DATETIME_SENT = str(f_meta1[0]) + ' ' + str(f_meta1[1])

        # Get TimeCode
        TIME_CODE = str(f_meta1[2]) + '.' + str(f_meta1[3])
        TIME_CODE2 = str(f_meta1[2]) + str(f_meta1[3])

        # Get DATETIME_RECEIVED
        DATETIME_RECEIVED = f_meta2

        return DATETIME_SENT, DATETIME_RECEIVED, TIME_CODE
    
    
    ##################################################################
    # Initialise Function
    ##################################################################
    
    # Define All Column Names (Especially for cases with an empty dataframe)
    col_names = [
                'ENTRY_CODE_SUFFIX', #00
                'ENTRY_CODE', #01
                'ALARM_ID', #02
                'USER_ID', #03
                'EQUIPMENT_NAME', #04
                'VALUE', #05
                'VALUE_STATE', #06
                'ACKNOWLEDGEMENT_REQUIRED', #07
                'SEVERITY', #08
                'HIDDEN', #09
                'THEME', #10
                'EQUIPMENT_DATE',  #11
                'ACQUISITION_DATE',  #12
                'SCS_TIME', #13
                'FUNCTIONAL_CATEGORY', #14
                'GEOGRAPHICAL_CATEGORY', #15
                'ENVIRONMENT', #16
                'USER1', #17
                'ASSET_ID_RAW', #18
                'ASSET_DESCRIPTION', #19
                'EVENT_DESCRIPTION', #20
                'EVENT_STATUS', #21
                'OPERATOR_INITIALS' #22
                ]
    
    # Define Attribute Names for Header (main body excluding metadata)
    headerList_core = [              
                      'ENTRY_CODE_RAW', #0
                      'ENTRY_CODE_SUFFIX', #1
                      'ENTRY_CODE', #2
                      'ALARM_ID', #3
                      'AUTO_ID', #4
                      'USER_ID', #5
                      'EQUIPMENT_NAME', #0
                      'VALUE', #7
                      'VALUE_STATE', #8
                      'ACKNOWLEDGEMENT_REQUIRED', #9
                      'SEVERITY', #10
                      'SHELVE', #11
                      'HIDDEN', #12
                      'THEME', #13
                      #'MESSAGE', #XX
                      'EQUIPMENT_DATE', #14
                      'EQUIPMENT_DATE0', #15
                      'ACQUISITION_DATE', #16
                      'ACQUISITION_DATE0', #17
                      'SCS_TIME', #18
                      'SCS_TIME0', #19
                      'FUNCTIONAL_CATEGORY', #20
                      'GEOGRAPHICAL_CATEGORY', #21
                      'ACKNOWLEDGEMENT_AUTOPOINTER', #22
                      'ENVIRONMENT', #23
                      'USER1', #24
                      'USER2', #25
                      'DSS_EVENT_TYPE' #26
                 ]
     
    ##################################################################
    # Ingest File
    ##################################################################
    
    # Define File Name
    fileName = fileInput
    
    # Read in file
    file = open(fileName, 'r')
        
    # Convert file contents to a list
    fileContents = list(file)

    # Close file
    file.close()

               
    ##################################################################
    # Extract Key Components in Ingested File
    ##################################################################
    
    # Extract row containing meta data of file sent datetime
    f_meta1 = fileContents[1]
    
    # Extract row containing meta data of file received datetime
    f_meta2 = fileContents[-1]
        
    # Remove redundant list elements
    del fileContents[0:3], fileContents[-1]
    
    # Error catch if this is the first log instance (empty log)
    if (f_meta1.find("first notification on online") > -1):
        # Create an Empty Dataframe with all the headers to match other dataframes to be appended
        df = pd.DataFrame(columns = col_names)
        
        # Delete redundant variables
        del f_meta1, f_meta2
    else:  
        # Load File
        df = pd.DataFrame(fileContents,columns=['rawData'])
        #print(df["rawData"][0])
        
        #print(ASSET_ID_RAW)
        #ASSET_ID_RAW = ASSET_ID_RAW.str.encode('ascii', 'ignore').str.decode() # TEST
        
        
        ##################################################################
        # Process Main Body (Exclude Free Text Message Field)
        ##################################################################
        
        # Exception handling for cases where the log entries are corrupted and split into 2 rows per entry
        # instead of the usual single row;
        # these cases tend to have a substring of unicode special characters marking the breakpoints

        # corruption check
        df["rawData"] = df["rawData"].str.strip() # remove non-printable char in string
        check_val_head = sum(df["rawData"].str.endswith("(.UàÑ"))
        check_val_tail = sum(df["rawData"].str.startswith("º÷"))
        
        print(check_val_head)
        print(check_val_tail)
        
        if (check_val_head == check_val_tail) & (check_val_head > 0) & (check_val_tail > 0):
            # If event log entry is corrupted, do this...
            
            # Separate breakpoint 1
            df_head = df.loc[df["rawData"].str.endswith("(.UàÑ")].copy()
            
            #print(df_head["rawData"][0])
            df_head["rawData"] = df_head["rawData"].str.split(pat = ".UàÑ", n = 1, expand = True)[0]
            df_head["rawData"] = df_head["rawData"].str.replace(pat = ";(", repl = "", regex = False)
            #df_head["rawData"] = df_head["rawData"].str.replace("(.UàÑ", regex = False)
            
            # Separate breakpoint 2
            df_tail = df.loc[df["rawData"].str.startswith("º÷")].copy()
            # drop first 3 char which has the unicode substring, direct targeting is not possible as that throws an error
            df_tail["rawData"] = df_tail["rawData"].str[3:] 
            
            # Reset index of dataframes
            df_head = df_head.copy().reset_index()
            df_tail = df_tail.copy().reset_index()
            
            # Join and repair corrupted entries which has been broken up into 2 rows
            df_head = df_head["rawData"] + df_tail["rawData"]

            # Filter out corrupted entires
            df = df.loc[(df["rawData"].str.endswith("(.UàÑ") == False)].copy()
            df = df.loc[(df["rawData"].str.startswith("º÷") == False)].copy()
            df = df.squeeze()
                        
            # Add back repaired corrupted entries
            df = df.append(df_head, ignore_index=True)
            df = pd.DataFrame(df, columns = ['rawData'])
            
            # Delete redundant variables
            del df_head, df_tail
            
            # Remove free text string "message" from log string
            df['text0'] = replaceTextBetween1_series(df["rawData"])
            
        else:
            # If event log entry is ok, do this...
            
            # Remove free text string "message" from log string
            df['text0'] = replaceTextBetween1_series(df["rawData"])
        
        # Delete redundant variables
        del check_val_head, check_val_tail
        
        # Delimit Columns
        df[headerList_core] = df['text0'].str.split(expand = True)

        # Merge Date and Time columns
        df['EQUIPMENT_DATE'] = df['EQUIPMENT_DATE'] + "." + df['EQUIPMENT_DATE0'].astype(str)
        df['ACQUISITION_DATE'] = df['ACQUISITION_DATE'] + "." + df['ACQUISITION_DATE0'].astype(str)
        df['SCS_TIME'] = df['SCS_TIME'] + "." + df['SCS_TIME0'].astype(str)
        
        # Extract Entry Code Suffix
        df['ENTRY_CODE_SUFFIX'] = df["ENTRY_CODE_RAW"].str.extract(r"\(""([+|=|-])")

        # Drop redundant columns
        df = df.drop(columns=['ENTRY_CODE_RAW', 
                              'EQUIPMENT_DATE0', 
                              'ACQUISITION_DATE0', 
                              'SCS_TIME0', 
                              'text0'])

        # Clean up values of each row by removing variable name prefixed to each value
        n = 3
        nEnd = df.shape[1]

        #df.iloc[::,n] = df.iloc[::,n].str.split(':', expand = True)[1]
        while (n < nEnd):
            df.iloc[::,n] = df.iloc[::,n].str.split(':', n = 1, expand = True)[1]
            n = n + 1

        # Delete redundant variables
        del n, nEnd

        # Drop redundant entries caused by entry deletion due to server memory limitations
        # Marked by "-" suffix in ["ENTRY_CODE_SUFFIX"]
        df = df.loc[df['ENTRY_CODE_SUFFIX'] != "-"].reset_index(drop=True)
        
        
        ##################################################################
        # Check if the remaining dataframe is empty or not
        ##################################################################

        if df.shape[0] == 0:
            #print("Empty Dataframe " + fileName)
            # Create an Empty Dataframe with all the headers to match other dataframes to be appended
            df  = pd.DataFrame(columns = col_names) 
        else:
            #print("Continue file processing")


            ##################################################################
            # Process Free Text Message Field
            ##################################################################

            # Remove free text string "message" from log string
            #del df['MESSAGE']
                        
            # Parse Free Text Message Log
            
            df["ASSET_ID_RAW"], df["ASSET_DESCRIPTION"], df["EVENT_DESCRIPTION"], df["EVENT_STATUS"], df["OPERATOR_INITIALS"] = parseMessageField(df["rawData"])
            
            ###############################################################
    
            # Reorder values of exception cases of EVENT_DESCRIPTION and ASSET_DESCRIPTION
            # EVENT_DESCRIPTION is prioritised
            try:
                df.loc[((df["EVENT_DESCRIPTION"] == None) | 
                       (df["EVENT_DESCRIPTION"] == "") | 
                       df["EVENT_DESCRIPTION"].isna()), 
                       "EVENT_DESCRIPTION"] = df["ASSET_DESCRIPTION"]

                df.loc[(df["EVENT_DESCRIPTION"] == df["ASSET_DESCRIPTION"]), 
                       "ASSET_DESCRIPTION"] = None                
            except:
                pass

            ###############################################################
            # Clean up for "GWS Broadcast" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("msg", na = False, regex = False)) & 
                       (df["EVENT_DESCRIPTION"].str.contains("Gws", na = False, regex = False)), "ASSET_DESCRIPTION"] = "GWS Broadcast"
            except:
                pass  

            ###############################################################
            # Clean up for "NelVisu" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("NelVisu", na = False, regex = False)), "ASSET_DESCRIPTION"] = "NelVisu"
            except:
                pass

            ###############################################################
            # Clean up for "Train Radio" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("TR___", na = False, regex = False)) &
                       (df["EVENT_DESCRIPTION"].str.contains("radio", na = False, regex = False)), "ASSET_DESCRIPTION"] = "Train Radio"
            except:
                pass

            ###############################################################
            # Clean up for "Trainborne Camera" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Trainborne Camera", na = False, regex = False)), "ASSET_DESCRIPTION"] = "Trainborne Camera"
            except:
                pass

            ###############################################################
            # Clean up for "Trainborne Quad" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Trainborne Quad", na = False, regex = False)), "ASSET_DESCRIPTION"] = "Trainborne Quad"
            except:
                pass

            ###############################################################
            # Clean up for "Tunnel Light" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Tunnel Light", na = False, regex = False)) & 
                        ((df["ASSET_DESCRIPTION"].isna()) | 
                         (df["ASSET_DESCRIPTION"] == None) | 
                         (df["ASSET_DESCRIPTION"] == "")), "ASSET_DESCRIPTION"] = "Tunnel LTG"
            except:
                pass   

            ###############################################################
            # Clean up for "Control Take Over" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Control Take Over for", na = False, regex = False)), "ASSET_DESCRIPTION"] = df["EVENT_DESCRIPTION"].str.extract(r"Control Take Over for (\w+) ")
                
            except:
                pass
            
            ###############################################################
            # Clean up for "Close Control" - Guess ASSET_DESCRIPTION based on EVENT_DESCRIPTION
            try:
                df.loc[(df["EVENT_DESCRIPTION"].str.contains("Close Control", na = False, regex = False)) & 
                      ((df["ASSET_DESCRIPTION"].isna()) | 
                       (df["ASSET_DESCRIPTION"] == None) | 
                       (df["ASSET_DESCRIPTION"] == "")
                      ), "ASSET_DESCRIPTION"] = "Traction Control"
            except:
                pass    

            ##################################################################
            # Format Data Type & Perform Final Clean Up
            ################################################################## 
            
            # Delete redundant variables (update 2.0)
            del df["rawData"]
            
            # Clean up operator initials (Update v2.0)
            try:
                df.loc[df["OPERATOR_INITIALS"].str.contains("(", na=False, regex=False), "OPERATOR_INITIALS"] = np.nan
            except:
                pass
            
            
            # Standardise null values
            df = df.replace("", np.nan).fillna(value=np.nan)          
            
            # Preliminary Formatting
            df = df.convert_dtypes()
            
            # Convert binary values to Boolean True/False
            try:
                df.loc[df["ACKNOWLEDGEMENT_REQUIRED"] == 0,"ACKNOWLEDGEMENT_REQUIRED"] = False
            except:
                pass
            try:
                df.loc[df["ACKNOWLEDGEMENT_REQUIRED"] == 1, "ACKNOWLEDGEMENT_REQUIRED"] = True
            except:
                pass
            
            try:
                df.loc[df["HIDDEN"] == 0, "HIDDEN"] = False
            except:
                pass            
            try:
                df.loc[df["HIDDEN"] == 1, "HIDDEN"] = True
            except:
                pass
            
            # Check data types
            df.dtypes# Preliminary Formatting
            df = df.convert_dtypes()

            # Format fields
            #df[''] = df[''].astype('')
            convert_dict = {'ALARM_ID': 'str',
                            'AUTO_ID': 'str',
                            'USER_ID': 'str',
                            'VALUE': 'str',
                            'VALUE_STATE': 'category',
                            'ACKNOWLEDGEMENT_REQUIRED': 'bool',
                            'SEVERITY': 'category',
                            'SHELVE': 'category',
                            'HIDDEN': 'bool',
                            'THEME': 'category',
                            'FUNCTIONAL_CATEGORY': 'category',
                            'GEOGRAPHICAL_CATEGORY': 'category',
                            'EQUIPMENT_DATE': 'float',
                            'ACQUISITION_DATE': 'float',
                            'SCS_TIME': 'float',
                            'ASSET_ID_RAW': 'str',
                            'ASSET_DESCRIPTION': 'str',
                            'EVENT_DESCRIPTION': 'str',
                            'EVENT_STATUS': 'category',
                            'OPERATOR_INITIALS': 'str'
                           }
            df = df.astype(convert_dict)

            # Delete redundant variables
            del convert_dict

            
            # This code block could be redundant as the fields are not being used
            # Format Unix Time Values to Datetime Format in Local Time (GMT+8)
            # 19 digits unix time (without decimal place): nanosecond resolution (ns)
            # 16 digits unix time (without decimal place): microsecond resolution (us)
            # 13 digits unix time (without decimal place): millisecond resolution (ms)
            # 10 digits unix time (without decimal place): second resolution (s)
            # The values seem to be 9 hours behind (standard unix time should be pegged to GMT)
            timeAdjust = 8
            df['EQUIPMENT_DATE'] = pd.to_datetime(df['EQUIPMENT_DATE'], unit='s') + pd.Timedelta(hours = timeAdjust)
            df['ACQUISITION_DATE'] = pd.to_datetime(df['ACQUISITION_DATE'], unit='s') + pd.Timedelta(hours = timeAdjust)
            df['SCS_TIME'] = pd.to_datetime(df['SCS_TIME'], unit='s') + pd.Timedelta(hours = timeAdjust)
            
            # Drop redundant columns
            df = df.drop(columns=[
                                    'AUTO_ID', 
                                    'SHELVE', 
                                    #'EQUIPMENT_DATE', 
                                    #'ACQUISITION_DATE', 
                                    #'SCS_TIME', 
                                    'ACKNOWLEDGEMENT_AUTOPOINTER', 
                                    'USER2', 
                                    'DSS_EVENT_TYPE'
                                   ])
            
    # Preliminary tagging of alarm events
    if (prefixInput == "AL") | (prefixInput == "TestAL"):
        df["isAlarm"] = True
    else:
        df["isAlarm"] = False
        
    # Convert boolean values to 1 and 0 for SQL Server Export
    if sqlServerExport == True:
        df["ACKNOWLEDGEMENT_REQUIRED"] = df["ACKNOWLEDGEMENT_REQUIRED"].astype(int)
        df["HIDDEN"] = df["HIDDEN"].astype(int)
        df["isAlarm"] = df["isAlarm"].astype(int)
    else:
        pass     
        

    ##################################################################
    # End
    ##################################################################
    
    return df

In [18]:
# Start clock to measure time taken
startTime = time.time()

testDF=cleaningScript_vector(testFile, prefix)

# End clock to measure time taken
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

0
0
Execution time in seconds: 0.05099773406982422
Execution time in minutes: 0.0008499622344970703
Execution time in hours: 1.4166037241617838e-05


In [19]:
testDF.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,isAlarm
0,=,53894,53894,0,:BGK:BMF:PMSA_0001:PLC1_0003,1,0,True,2,True,0,2021-12-01 09:00:09.830429952,2021-12-01 09:00:09.830429952,2021-12-01 09:00:09.830429952,51,17,OCCCMS,1,SCS/BGK/B1/PLC03,BGK ISCS PLC 3,Mux Selection,NORMAL,<NA>,True


In [20]:
print(testDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ENTRY_CODE_SUFFIX         1 non-null      string        
 1   ENTRY_CODE                1 non-null      string        
 2   ALARM_ID                  1 non-null      object        
 3   USER_ID                   1 non-null      object        
 4   EQUIPMENT_NAME            1 non-null      string        
 5   VALUE                     1 non-null      object        
 6   VALUE_STATE               1 non-null      category      
 7   ACKNOWLEDGEMENT_REQUIRED  1 non-null      bool          
 8   SEVERITY                  1 non-null      category      
 9   HIDDEN                    1 non-null      bool          
 10  THEME                     1 non-null      category      
 11  EQUIPMENT_DATE            1 non-null      datetime64[ns]
 12  ACQUISITION_DATE          

In [21]:
testDF["VALUE"]

0    1
Name: VALUE, dtype: object

In [22]:
testFile2 = fileList[1]
testFile2

# Start clock to measure time taken
startTime = time.time()

testDF=cleaningScript_vector(testFile2, prefix)

# End clock to measure time taken
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print('Execution time in minutes: ' + str(executionTime/60))
print('Execution time in hours: ' + str(executionTime/60/60))

0
0
Execution time in seconds: 0.05299973487854004
Execution time in minutes: 0.0008833289146423339
Execution time in hours: 1.4722148577372232e-05


In [23]:
testFile2

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\AlarmLogs\\AlarmListTest_1638320436_251896'

In [24]:
testFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\sample_data_test\\batch 002\\AlarmLogs\\AlarmListTest_1638320410_254559'

In [25]:
testDF.head()

,ENTRY_CODE_SUFFIX,ENTRY_CODE,ALARM_ID,USER_ID,EQUIPMENT_NAME,VALUE,VALUE_STATE,ACKNOWLEDGEMENT_REQUIRED,SEVERITY,HIDDEN,THEME,EQUIPMENT_DATE,ACQUISITION_DATE,SCS_TIME,FUNCTIONAL_CATEGORY,GEOGRAPHICAL_CATEGORY,ENVIRONMENT,USER1,ASSET_ID_RAW,ASSET_DESCRIPTION,EVENT_DESCRIPTION,EVENT_STATUS,OPERATOR_INITIALS,isAlarm
0,=,53983,53983,0,:BGK:BMF:PMSA_0001:PLC1_0002,0,1,True,2,True,0,2021-12-01 09:00:36.869270016,2021-12-01 09:00:36.869270016,2021-12-01 09:00:36.869270016,51,17,OCCCMS,1,SCS/BGK/B1/PLC02,BGK ISCS PLC 2,Mux Selection,FAULT,<NA>,True


In [26]:
print(testDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ENTRY_CODE_SUFFIX         1 non-null      string        
 1   ENTRY_CODE                1 non-null      string        
 2   ALARM_ID                  1 non-null      object        
 3   USER_ID                   1 non-null      object        
 4   EQUIPMENT_NAME            1 non-null      string        
 5   VALUE                     1 non-null      object        
 6   VALUE_STATE               1 non-null      category      
 7   ACKNOWLEDGEMENT_REQUIRED  1 non-null      bool          
 8   SEVERITY                  1 non-null      category      
 9   HIDDEN                    1 non-null      bool          
 10  THEME                     1 non-null      category      
 11  EQUIPMENT_DATE            1 non-null      datetime64[ns]
 12  ACQUISITION_DATE          

## Set Up SQL Server Database & Functions
The goal is to dump the pre-processed file into a SQL Server Database where further operations may then be made to perform the alarm tagging and nuisance event tagging. But first we will need to set up the SQL Server Database.

** Key tables **
1. EventList Table
2. AlarmList Table
3. Output_AlarmTagged Table
4. Output_TagComplete Table

In [27]:
# Open a database connection to target database
# All subsequent functions will depend on this connection
# Remember to close connection when done
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=SBSR-RD-0K00200;'
                      'Database=IAMS_DBtest;'
                      'Trusted_Connection=yes;')

# Define Server Parameters to Initiate Connection Engine via SQL Alchemy
# This has the same values as the connection request
# Only if one uses the windows authentification method
# Otherwise, one will need to define "UID" (user ID) + "PWD" (password)
serverParams = urllib.parse.quote_plus('Driver={SQL Server};'
                                       'Server=SBSR-RD-0K00200;'
                                       'Database=IAMS_DBtest;'
                                       'Trusted_Connection=yes;'
                                       #"UID=user;"
                                       #"PWD=password"
                                      )

# Create cursor to work in database
# SQL auto commits transactions
cursor = conn.cursor()

In [28]:
# Initialise tables (empty)
if initDB == True:
    # Initialise eventList Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='eventList')
                    CREATE TABLE eventList (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED int null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        ASSET_DESC_CAT varchar(255) null,
                        EVENT_DESC_CAT varchar(255) null,
                        TrainID varchar(255) null,
                        CarID varchar(255) null,
                        ServiceID varchar(255) null,
                        AssetClass varchar(255) null,
                        AssetSubClass varchar(255) null,
                        DATETIME_SENT datetime null,
                        DATETIME_RECEIVED datetime null,
                        TIME_CODE datetime null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    
    # Initialise alarmList Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='alarmList')
                    CREATE TABLE alarmList (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED varchar(255) null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    
    # Initialise Output_AlarmTagged Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='Output_AlarmTagged')
                    CREATE TABLE Output_AlarmTagged (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED int null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        ASSET_DESC_CAT varchar(255) null,
                        EVENT_DESC_CAT varchar(255) null,
                        TrainID varchar(255) null,
                        CarID varchar(255) null,
                        ServiceID varchar(255) null,
                        AssetClass varchar(255) null,
                        AssetSubClass varchar(255) null,
                        DATETIME_SENT datetime null,
                        DATETIME_RECEIVED datetime null,
                        TIME_CODE datetime null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    
    # Initialise Output_TagComplete
    cursor.execute('''
                if not exists (select * from sys.tables where name='Output_TagComplete')
                    CREATE TABLE Output_TagComplete (
                        ENTRY_CODE_SUFFIX varchar(255) null, 
                        ENTRY_CODE varchar(255) null,
                        ALARM_ID varchar(255) null,
                        USER_ID varchar(255) null,
                        EQUIPMENT_NAME varchar(255) null,
                        VALUE varchar(255) null,
                        VALUE_STATE varchar(255) null,
                        ACKNOWLEDGEMENT_REQUIRED int null,
                        SEVERITY varchar(255) null,
                        HIDDEN int null,
                        THEME varchar(255) null,
                        EQUIPMENT_DATE datetime null,
                        ACQUISITION_DATE datetime null,
                        SCS_TIME datetime null,
                        FUNCTIONAL_CATEGORY varchar(255) null,
                        GEOGRAPHICAL_CATEGORY varchar(255) null,
                        ENVIRONMENT varchar(255) null,
                        USER1 varchar(255) null,
                        ASSET_ID_RAW varchar(255) null,
                        ASSET_DESCRIPTION varchar(255) null,
                        EVENT_DESCRIPTION varchar(255) null,
                        EVENT_STATUS varchar(255) null,
                        OPERATOR_INITIALS varchar(255) null,
                        ASSET_DESC_CAT varchar(255) null,
                        EVENT_DESC_CAT varchar(255) null,
                        TrainID varchar(255) null,
                        CarID varchar(255) null,
                        ServiceID varchar(255) null,
                        AssetClass varchar(255) null,
                        AssetSubClass varchar(255) null,
                        DATETIME_SENT datetime null,
                        DATETIME_RECEIVED datetime null,
                        TIME_CODE datetime null,
                        isAlarm int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE()),
                        RepeatAlarm int null,
                        AltAlarm2 int null,
                        AltAlarm3 int null,
                        NuisanceAlarm int null
                    )
                    ''')
    
    # Initialise TestValues_Master Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='TestValues_Master')
                    CREATE TABLE TestValues_Master (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    # Initialise TestValues_Alarm Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='TestValues_Alarm')
                    CREATE TABLE TestValues_Alarm (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    # Initialise Test_AlarmTagged Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='Test_AlarmTagged')
                    CREATE TABLE Test_AlarmTagged (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE())
                    )
                    ''')
    # Initialise Test_Final Table
    cursor.execute('''
                if not exists (select * from sys.tables where name='Test_Final')
                    CREATE TABLE Test_Final (
                        eventID int null, 
                        DATEANDTIME datetime null,
                        eventDesc text null,
                        TnF int null,
                        blanCol int null,
                        DATETIME_LOADED DATETIME NULL DEFAULT(GETDATE()),
                        RepeatAlarm int null, 
                        ToggleEventA int null, 
                        ToggleEventB int null, 
                        NuisanceAlarm int null
                    )
                    ''')
    
    
    conn.commit()
else:
    pass

In [29]:
# Function to Inspect All Tables
def list_dbTables():
    # Check list of tables in db
    cursor.execute("SELECT table_name FROM INFORMATION_SCHEMA.TABLES ")
    print(cursor.fetchall())

# List tables in database
# Note: %memit" prefix is used to log peak memory uage
list_dbTables()

[('eventList', ), ('alarmList', ), ('Output_AlarmTagged', ), ('Output_TagComplete', ), ('AppendTest', ), ('CompareTest', )]


In [30]:
# Function to list contents in target table
def list_tableContents(targetTable, rowLimit = 10):
    # Gets first n rows (rowLimit) from target table sorted by datetime (oldest entry first)
    # Not allowed to get all values as the table size can be huge
    cursor.execute(f"SELECT top {rowLimit} * FROM {targetTable} ORDER BY 'DATEANDTIME' ASC")
    results = cursor.fetchall()
    print(targetTable, "Contents")
    counter = 0
    print("Table Values")
    print("---START---")
    for row in results:
        counter=counter+1
        print(counter, row)
    print("---END---")

# Define the target table
# This table will be where the cleaned data would be saved to 
# and manipulated for future operations
targetTable = "AppendTest"

# Inspect target table
list_tableContents(targetTable)

AppendTest Contents
Table Values
---START---
---END---


In [31]:
# Function to list contents in target table
def list_tableColDtype(targetTable):
    cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM information_schema.columns where TABLE_NAME = '{targetTable}'")
    results = cursor.fetchall()
    print(targetTable, "Contents")
    counter = 0
    print("Table Column Data Type")
    print("---START---")
    for row in results:
        counter=counter+1
        print(counter, row)
    print("---END---")

# Inspect target table
targetTable = "AppendTest"
list_tableColDtype(targetTable)

AppendTest Contents
Table Column Data Type
---START---
1 ('eventID', 'int')
2 ('DATEANDTIME', 'datetime')
3 ('eventDesc', 'text')
4 ('TnF', 'int')
5 ('blanCol', 'int')
6 ('DATETIME_LOADED', 'datetime')
---END---


In [32]:
# Function to Create a dataframe with dummy data for testing purposes
def createDummyDataDF(inputData = "A"):
    if (inputData == "A"):
        dummyData = {
                     'eventID': [1, 2, 3],
                     'DATEANDTIME': [1644223829, 1644310229, 1644396629],
                     'eventDesc': ['Alpha', 'Bravo', 'Charlie'],
                     'TnF': [1, 1, 1] 
                    }
    else:
        dummyData = {
                     'eventID': [4, 5, 6],
                     'DATEANDTIME': [1644223829, 1644310229, 1644396629],
                     'eventDesc': ['Delta', 'Echo', 'Foxtrot'],
                     'TnF': [0, 0, 0] 
                    }
    
    df = pd.DataFrame(dummyData)
    df['DATEANDTIME'] = pd.to_datetime(df['DATEANDTIME'], unit='s')

    # To get time in seconds resolution if it comes in higher resolutions
    # Not required
    #df['TIME_S'] = df['DATEANDTIME'].dt.floor("s").dt.time

    return df

# Generate test dataframe in testing mode
if (testMode == True):
    # Create a dummy dataframe for testing purposes
    testDF = createDummyDataDF()
    print("Dataframe columns would have the same data type as SQL table values")
    testDF.info()
else:
    print("Test script skipped")

Dataframe columns would have the same data type as SQL table values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   eventID      3 non-null      int64         
 1   DATEANDTIME  3 non-null      datetime64[ns]
 2   eventDesc    3 non-null      object        
 3   TnF          3 non-null      int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 224.0+ bytes


In [33]:
# Inspect test dataframe
if (testMode == True):
    # Inspect Data
    print(testDF.head())
else:
    print("Test script skipped")

   eventID         DATEANDTIME eventDesc  TnF
0        1 2022-02-07 08:50:29     Alpha    1
1        2 2022-02-08 08:50:29     Bravo    1
2        3 2022-02-09 08:50:29   Charlie    1


In [34]:
# Function to append data to database as an entire dataframe
def appendData(tableName, inputDF, serverParams):    
    # Create connection engine 
    # Default connection function only works for SQLite
    engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(serverParams))
    # Append data
    inputDF.to_sql(tableName, con=engine, if_exists="append", index = False)

In [35]:
# Test data append function
if (testMode == True):
    # Test if append dataframe function works
    appendData("AppendTest", testDF, serverParams)
    list_tableContents("AppendTest")
else:
    print("Test script skipped")

AppendTest Contents
Table Values
---START---
1 (1, datetime.datetime(2022, 2, 7, 8, 50, 29), 'Alpha', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 33, 297000))
2 (2, datetime.datetime(2022, 2, 8, 8, 50, 29), 'Bravo', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 33, 297000))
3 (3, datetime.datetime(2022, 2, 9, 8, 50, 29), 'Charlie', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 33, 297000))
---END---


In [36]:
# Function to delete last n rows in table sorted by datetime (oldest first)
def delDataNRow(targetTable, nRow=3):
    cursor.execute(f"WITH CTE AS (SELECT TOP {nRow} * FROM {targetTable} ORDER BY DATEANDTIME DESC) DELETE FROM CTE")
    conn.commit()
    
# Function to delete all rows in table
def delDataAll(targetTable):
    cursor.execute(f"DELETE FROM {targetTable}")
    conn.commit()

In [37]:
# Test data delete latest nRows function
if (testMode == True):
    delDataNRow(targetTable, nRow=2)
    list_tableContents(targetTable)
else:
    print("Test script skipped")

AppendTest Contents
Table Values
---START---
1 (1, datetime.datetime(2022, 2, 7, 8, 50, 29), 'Alpha', 1, None, datetime.datetime(2022, 4, 7, 11, 11, 33, 297000))
---END---


In [38]:
# Test data delete all function
if (resetTableData == True):
    delDataAll(targetTable)
    list_tableContents(targetTable)
else:
    print("Test script skipped")

AppendTest Contents
Table Values
---START---
---END---


In [39]:
# Delete redundant variables used in test
if (resetTableData == True):
    del testDF
else:
    print("Test script skipped")

## Push Raw Logs to SQL Server

In [40]:
# Define function to transfer files which has been processed successfully to an archive folder
def fileTransfer(file_name, dst_folder, src_folder=""):
    # If current folder is the source directory for the files, leave src_folder empty
    # check if file exist in destination
    if os.path.exists(dst_folder + file_name):
        # Split name and file type extension
        data = os.path.splitext(file_name)
        only_name = data[0]
        extension = data[1]
        # Adding the new name
        new_base = only_name + '_new' + extension
        # construct full file path
        new_name = os.path.join(dst_folder, new_base)
        # move file
        shutil.move(src_folder + file_name, new_name)
    else:
        shutil.move(src_folder + file_name, dst_folder + file_name)
    print(file_name, "Transferred to destination folder", dst_folder)

In [41]:
# Get timestamp of operation start:
dateTimeStartOp = dt.datetime.now()
print("Start Operation")
print(dateTimeStartOp)

# Get list of files to initialise
fileList = os.listdir()
# Ignore temp files (large files not fully downloaded)
fileList = [filteredList for filteredList in fileList if "~$" not in filteredList]
fileList = [filteredList for filteredList in fileList if ".temp" not in filteredList]
fileList = [filteredList for filteredList in fileList if ".tmp" not in filteredList]

# Define Archive Folder Location and Target table
if (prefix == 'TestAE'):
    targetTable = "eventList"
    archiveFolderLoc = "../EventLogs_archive/"
else:
    targetTable = "alarmList"
    archiveFolderLoc = "../AlarmLogs_archive/"
    
# Define wait time for new files to be added
# If there are no more files within directory
# Else the script will be terminated
# Time is in seconds
waitTimeInterval = 1
waitTimeLimit = 10

while (len(fileList) > 0):
    # Initiate batch run  
    print("Start batch run")
    # Get timestamp of batch run start:
    dateTimeStartBatch = dt.datetime.now()
    print(dateTimeStartBatch)
    
    for file in fileList:
        # Get Oldest File in Directory in a First in First Out (FIFO) manner as an input
        inputFile = min(fileList, key=os.path.getctime)
        
        #############################################
        # Process file as a cleaned dataframe - START
        #############################################
        
        # Get timestamp of run start:
        print("Commence processing of", inputFile)
        dateTimeStartRun = dt.datetime.now()
        print(dateTimeStartRun)
        
        # Ingest log files as a dataframe for further processing
        df = cleaningScript_vector(inputFile, prefix, True)
        
        #############################################
        # Process file as a cleaned dataframe - END
        #############################################
        
        # Append data to SQL DB
        appendData(targetTable, df, serverParams)

        # Archive processed file
        fileTransfer(inputFile, archiveFolderLoc)
        #print(inputFile, "archived")
        
        # Update list of files
        fileList = os.listdir()
        # Ignore temp files (large files not fully downloaded)
        fileList = [filteredList for filteredList in fileList if "~$" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".temp" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".tmp" not in filteredList]
        
        # Get timestamp of run end: 
        dateTimeEndRun = dt.datetime.now()
        print("Run Tlapsed Time")
        print(dateTimeEndRun - dateTimeStartRun)
        del dateTimeEndRun, dateTimeStartRun
        
    print("Batch run completed")
    # Get timestamp of batch run end:
    dateTimeEndBatch = dt.datetime.now()
    print(dateTimeEndBatch)
    print("Batch Elapsed Time")
    print(dateTimeEndBatch - dateTimeStartBatch)
    del dateTimeEndBatch, dateTimeStartBatch
    
    # Check if new files are added to restart process
    timeCounter = 0
    while (len(fileList) == 0):
        # When initial batch run is completed check again for newly added files
        # This is to account for any lag or latency in the file transfer
        # n second(s) timer delay between batches
        time.sleep(waitTimeInterval)
        timeCounter = timeCounter + waitTimeInterval
        
        # Update list of files
        fileList = os.listdir()
        # Ignore temp files (large files not fully downloaded)
        fileList = [filteredList for filteredList in fileList if "~$" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".temp" not in filteredList]
        fileList = [filteredList for filteredList in fileList if ".tmp" not in filteredList]

        # Terminate script if wait too long
        if (timeCounter > waitTimeLimit):
            break
        
    
print("Script Terminated")
dateTimeEndOp = dt.datetime.now()
print(dateTimeEndOp)
print("Operation Elapsed Time")
print(dateTimeEndOp - dateTimeStartOp)

Start Operation
2022-04-07 11:11:33.410376
Start batch run
2022-04-07 11:11:33.413377
Commence processing of AlarmListTest_1638320410_254559
2022-04-07 11:11:33.445376
0
0
AlarmListTest_1638320410_254559 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.074000
Commence processing of AlarmListTest_1638320436_251896
2022-04-07 11:11:33.551376
0
0
AlarmListTest_1638320436_251896 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:04.151723
Commence processing of AlarmListTest_1638320437_252131
2022-04-07 11:11:37.740101
0
0
AlarmListTest_1638320437_252131 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.117000
Commence processing of AlarmListTest_1638320443_251730
2022-04-07 11:11:37.890129
0
0
AlarmListTest_1638320443_251730 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.078972
Commence processing of AlarmListTest_1638320444_252726
2022-04-07 11:11:38.003101
0
0
Alar

AlarmListTest_1638320608_251982 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.072000
Commence processing of AlarmListTest_1638320616_252463
2022-04-07 11:12:06.243464
0
0
AlarmListTest_1638320616_252463 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.085000
Commence processing of AlarmListTest_1638320617_252678
2022-04-07 11:12:06.361464
0
0
AlarmListTest_1638320617_252678 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.086001
Commence processing of AlarmListTest_1638320619_252364
2022-04-07 11:12:06.484465
0
0
AlarmListTest_1638320619_252364 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.111000
Commence processing of AlarmListTest_1638320621_254256
2022-04-07 11:12:06.634464
0
0
AlarmListTest_1638320621_254256 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.052001
Commence processing of AlarmListTest_1638320629_254212
2

0
0
AlarmListTest_1638320758_252605 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.063003
Commence processing of AlarmListTest_1638320759_252527
2022-04-07 11:12:10.979876
0
0
AlarmListTest_1638320759_252527 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.062999
Commence processing of AlarmListTest_1638320760_252540
2022-04-07 11:12:11.069875
0
0
AlarmListTest_1638320760_252540 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.072970
Commence processing of AlarmListTest_1638320768_251494
2022-04-07 11:12:11.173845
0
0
AlarmListTest_1638320768_251494 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.069001
Commence processing of AlarmListTest_1638320769_251418
2022-04-07 11:12:11.272842
0
0
AlarmListTest_1638320769_251418 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.066001
Commence processing of AlarmListTest_1638320771_2554

AlarmListTest_1638320943_253996 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.068004
Commence processing of AlarmListTest_1638320949_252868
2022-04-07 11:12:14.940843
0
0
AlarmListTest_1638320949_252868 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.079017
Commence processing of AlarmListTest_1638320955_252795
2022-04-07 11:12:15.048875
0
0
AlarmListTest_1638320955_252795 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.068969
Commence processing of AlarmListTest_1638320956_254024
2022-04-07 11:12:15.148845
0
0
AlarmListTest_1638320956_254024 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.079001
Commence processing of AlarmListTest_1638320960_251688
2022-04-07 11:12:15.256846
0
0
AlarmListTest_1638320960_251688 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.083999
Commence processing of AlarmListTest_1638320961_251633
2

Run Tlapsed Time
0:00:00.070970
Commence processing of AlarmListTest_1638321136_254318
2022-04-07 11:12:26.712641
0
0
AlarmListTest_1638321136_254318 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.060970
Commence processing of AlarmListTest_1638321137_254332
2022-04-07 11:12:26.798646
0
0
AlarmListTest_1638321137_254332 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.060965
Commence processing of AlarmListTest_1638321140_252193
2022-04-07 11:12:26.883610
0
0
AlarmListTest_1638321140_252193 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.061001
Commence processing of AlarmListTest_1638321141_252114
2022-04-07 11:12:26.976613
0
0
AlarmListTest_1638321141_252114 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.060997
Commence processing of AlarmListTest_1638321150_251965
2022-04-07 11:12:27.061610
0
0
AlarmListTest_1638321150_251965 Transferred to destination

...

Run Tlapsed Time
0:00:00.097999
Commence processing of AlarmListTest_1638323118_251682
2022-04-07 11:13:13.832044
0
0
AlarmListTest_1638323118_251682 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.092001
Commence processing of AlarmListTest_1638323119_252804
2022-04-07 11:13:13.934044
0
0
AlarmListTest_1638323119_252804 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.041030
Commence processing of AlarmListTest_1638323145_252079
2022-04-07 11:13:13.984047
0
0
AlarmListTest_1638323145_252079 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.076001
Commence processing of AlarmListTest_1638323153_251932
2022-04-07 11:13:14.070044
0
0
AlarmListTest_1638323153_251932 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.080002
Commence processing of AlarmListTest_1638323154_251913
2022-04-07 11:13:14.160044
0
0
AlarmListTest_1638323154_251913 Transferred to destination

AlarmListTest_1638323292_252218 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.071000
Commence processing of AlarmListTest_1638323293_252318
2022-04-07 11:13:17.361044
0
0
AlarmListTest_1638323293_252318 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.067004
Commence processing of AlarmListTest_1638323294_252350
2022-04-07 11:13:17.435080
0
0
AlarmListTest_1638323294_252350 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.065964
Commence processing of AlarmListTest_1638323320_251686
2022-04-07 11:13:17.508044
0
0
AlarmListTest_1638323320_251686 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.070004
Commence processing of AlarmListTest_1638323322_251723
2022-04-07 11:13:17.585048
0
0
AlarmListTest_1638323322_251723 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.071001
Commence processing of AlarmListTest_1638323325_251673
2

0
0
AlarmListTest_1638323444_254272 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.054969
Commence processing of AlarmListTest_1638323449_254228
2022-04-07 11:13:20.813044
0
0
AlarmListTest_1638323449_254228 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.087002
Commence processing of AlarmListTest_1638323451_255327
2022-04-07 11:13:20.905046
0
0
AlarmListTest_1638323451_255327 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.037002
Commence processing of AlarmListTest_1638323463_253012
2022-04-07 11:13:20.947079
0
0
AlarmListTest_1638323463_253012 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.066966
Commence processing of AlarmListTest_1638323464_252946
2022-04-07 11:13:21.019048
0
0
AlarmListTest_1638323464_252946 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.066996
Commence processing of AlarmListTest_1638323465_2519

AlarmListTest_1638323676_251787 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:04.108001
Commence processing of AlarmListTest_1638323685_251635
2022-04-07 11:13:48.231918
0
0
AlarmListTest_1638323685_251635 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:04.143001
Commence processing of AlarmListTest_1638323686_251955
2022-04-07 11:13:52.377918
0
0
AlarmListTest_1638323686_251955 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.070000
Commence processing of AlarmListTest_1638323690_252571
2022-04-07 11:13:52.451937
0
0
AlarmListTest_1638323690_252571 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:04.139981
Commence processing of AlarmListTest_1638323691_253790
2022-04-07 11:13:56.594918
0
0
AlarmListTest_1638323691_253790 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.083001
Commence processing of AlarmListTest_1638323715_252103
2

AlarmListTest_1638323898_251588 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.078000
Commence processing of AlarmListTest_1638323907_254440
2022-04-07 11:14:20.184255
0
0
AlarmListTest_1638323907_254440 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.089000
Commence processing of AlarmListTest_1638323913_254252
2022-04-07 11:14:20.275255
0
0
AlarmListTest_1638323913_254252 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.081004
Commence processing of AlarmListTest_1638323922_251997
2022-04-07 11:14:20.357260
0
0
AlarmListTest_1638323922_251997 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.084998
Commence processing of AlarmListTest_1638323933_251971
2022-04-07 11:14:20.444259
0
0
AlarmListTest_1638323933_251971 Transferred to destination folder ../AlarmLogs_archive/
Run Tlapsed Time
0:00:00.084999
Commence processing of AlarmListTest_1638323934_251838
2

In [42]:
# Close Connection to Database
conn.close()